# <font color='blue'> <center> ScalaAI</center> </font> 

## <center> Escalação Automática e Otimizada de Times do CartolaFC - 2021 </center> 
### Versão Teste - Escalação na Formação 4-3-3

<img src = "https://github.com/lucianogarim/ScalaAI/blob/main/sca.jpg?raw=true" height = "350" width="800">

Importação dos pacotes necessários

In [1]:
%pylab
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


Leitura dos dados da API do CartolaFC

In [2]:
response = requests.get('https://api.cartolafc.globo.com/atletas/mercado')
data = json.loads(response.text)
atletas = pd.DataFrame(data['atletas'])
times = pd.DataFrame(data['clubes']).T

Remove as colunas desnecessárias

In [3]:
atletas.drop(['atleta_id','scout', 'slug', 'apelido_abreviado', 'nome', 'foto'], axis=1, inplace=True)
times.drop(['abreviacao', 'escudos', 'nome_fantasia'], axis=1, inplace=True)
atletas = atletas[['apelido', 'clube_id', 'posicao_id', 'preco_num', 'pontos_num', 'variacao_num', 
                   'media_num', 'rodada_id', 'jogos_num','status_id']]

Organizando os dados

In [4]:
atletas.posicao_id = atletas.posicao_id.map({1:'Goleiro', 2:'Lateral', 3:'Zagueiro', 4:'Meia', 5:'Atacante', 6:'Tecnico'})
times_id = times['id'].to_list()
times_nome = times['nome'].to_list()
dictionary = dict(zip(times_id, times_nome))
atletas.clube_id = atletas.clube_id.map(dictionary)
#renomeia as colunas
atletas.rename(columns={'clube_id': 'clube', 'posicao_id': 'posicao', 'preco_num': 'preco', 'pontos_num': 'pontos',
                        'variacao_num': 'variacao', 'media_num': 'media', 'rodada_id': 'rodada',
                        'jogos_num': 'jogos'}, inplace=True)

Considerando a escalação de apenas jogadores prováveis

In [5]:
atletas = atletas[atletas.status_id==7].reset_index()
atletas.drop(['index','status_id'], axis=1, inplace=True)
atletas.reset_index()

,index,apelido,clube,posicao,preco,pontos,variacao,media,rodada,jogos
0,0,Fagner,Corinthians,Lateral,11.99,0,-0.06,3.98,21,20
1,1,Weverton,Palmeiras,Goleiro,15.79,0,1.78,3.85,21,10
2,2,Thiago Heleno,Athlético-PR,Zagueiro,9.04,0,0.00,2.49,21,15
3,3,Richard,Ceará,Goleiro,8.31,0,1.31,4.31,21,16
4,4,Ferreira,Grêmio,Atacante,10.44,0,-0.25,4.94,21,11
...,...,...,...,...,...,...,...,...,...,...
210,210,Michel,Juventude,Lateral,3.52,0,-0.07,1.94,21,16
211,211,Pepê,Cuiabá,Meia,9.37,0,0.00,3.70,21,16
212,212,Marlon,América-MG,Lateral,7.01,0,1.33,6.67,21,6
213,213,Vina,Ceará,Meia,11.27,0,0.78,1.80,21,18


# Algoritmo Genético para Escalação Automática

In [6]:
dados = pd.DataFrame(atletas[['apelido','media','posicao','preco','pontos','jogos']])
dados.index = dados.index.set_names(['id'])
dados = dados.reset_index()

best = 0
Goleiro = 1
Zagueiro = 2
Zagueiro = 3
Lateral = 4
Lateral = 5
Meia = 6
Meia = 7
Meia = 8
Atacante = 9
Atacante = 10
Atacante = 11
Tecnico = 12

peso_media = 0.5
peso_jogos = 0.25
peso_pontos = 0.25

score_atleta = ((dados.media/dados.media.max()) * peso_media + (dados.jogos/dados.jogos.max()) * peso_jogos 
                + (dados.pontos/dados.pontos.max()) * peso_pontos)
dados['score_atleta'] = score_atleta


# População inicial
populacao = list(range(30))
for i in range(0,30):
    populacao[i]=[random.choice(dados.loc[dados.posicao=='Goleiro']['id']),
                  random.choice(dados.loc[dados.posicao=='Zagueiro']['id']),
                  random.choice(dados.loc[dados.posicao=='Zagueiro']['id']),
                  random.choice(dados.loc[dados.posicao=='Lateral']['id']),
                  random.choice(dados.loc[dados.posicao=='Lateral']['id']),
                  random.choice(dados.loc[dados.posicao=='Meia']['id']),
                  random.choice(dados.loc[dados.posicao=='Meia']['id']),
                  random.choice(dados.loc[dados.posicao=='Meia']['id']),
                  random.choice(dados.loc[dados.posicao=='Atacante']['id']),
                  random.choice(dados.loc[dados.posicao=='Atacante']['id']),
                  random.choice(dados.loc[dados.posicao=='Atacante']['id']),
                  random.choice(dados.loc[dados.posicao=='Tecnico']['id'])]
a = []
c = []
control = 0
while (control==0):
    # Função de custo
    for k in range(200):
        cartoletas = 105
        for i in range(0,shape(populacao)[0]):
            pontuacao = dados.iloc[populacao[i]].score_atleta.sum()/12
            c.append(pontuacao)
            if dados.iloc[populacao[i]].preco.sum() > cartoletas:
                populacao[i].append(100)
            else:
                populacao[i].append(pontuacao)
                if populacao[i][-1]>best:
                    best = populacao[i][-1]
                    melhor =  populacao[i]
                    a.append(populacao[i][-1])
    # Seleção
        caixa = populacao
        selecao = []
        for i in range(0,12):
            candidato1 = caixa[random.choice(len(caixa))]
            candidato2 = candidato1
            while (candidato1[0]==candidato2[0]):
                candidato2 = caixa[random.choice(len(caixa))]   #até aqui ok

            if candidato1[-1]>candidato2[-1]:
                selecao.append(candidato1)
                caixa.remove(candidato1)
            else:
                selecao.append(candidato2)
                caixa.remove(candidato2)
    # Reprodução
        geracao = []
        while (len(selecao)!=0):
            candidato1 = selecao[random.choice(len(selecao))]
            candidato2 = candidato1
            while (candidato1==candidato2):
                candidato2 = selecao[random.choice(len(selecao))] 
            filho1 = candidato1[0:3]+candidato2[3:6]+candidato1[6:-1]
            pos = random.choice(len(filho1))
            if pos==Goleiro:
                filho1[pos] = random.choice(dados[dados.posicao=='Goleiro'].id)
            elif pos==Zagueiro:
                filho1[pos] = random.choice(dados[dados.posicao=='Zagueiro'].id)
            elif pos==Lateral:
                filho1[pos] = random.choice(dados[dados.posicao=='Lateral'].id)
            elif pos==Meia:
                filho1[pos] = random.choice(dados[dados.posicao=='Meia'].id)
            elif pos==Atacante:
                filho1[pos] = random.choice(dados[dados.posicao=='Atacante'].id)
            else:
                filho1[pos] = random.choice(dados[dados.posicao=='Tecnico'].id)

            filho2 = candidato2[0:3]+candidato1[3:6]+candidato2[6:-1]
            pos = random.choice(len(filho2))
            if pos==Goleiro:
                filho2[pos] = random.choice(dados[dados.posicao=='Goleiro'].id)
            elif pos==Zagueiro:
                filho2[pos] = random.choice(dados[dados.posicao=='Zagueiro'].id)
            elif pos==Lateral:
                filho2[pos] = random.choice(dados[dados.posicao=='Lateral'].id)
            elif pos==Meia:
                filho2[pos] = random.choice(dados[dados.posicao=='Meia'].id)
            elif pos==Atacante:
                filho2[pos] = random.choice(dados[dados.posicao=='Atacante'].id)
            else:
                filho2[pos] = random.choice(dados[dados.posicao=='Tecnico'].id)

            selecao.remove(candidato1)
            selecao.remove(candidato2)
            geracao.append(candidato1[:-1])
            geracao.append(candidato2[:-1])
    # Completa população
        populacao = list(range(18))
        for i in range(0,18):
            populacao[i]=[random.choice(dados.loc[dados.posicao=='Goleiro']['id']),
                          random.choice(dados.loc[dados.posicao=='Zagueiro']['id']),
                          random.choice(dados.loc[dados.posicao=='Zagueiro']['id']),
                          random.choice(dados.loc[dados.posicao=='Lateral']['id']),
                          random.choice(dados.loc[dados.posicao=='Lateral']['id']),
                          random.choice(dados.loc[dados.posicao=='Meia']['id']),
                          random.choice(dados.loc[dados.posicao=='Meia']['id']),
                          random.choice(dados.loc[dados.posicao=='Meia']['id']),
                          random.choice(dados.loc[dados.posicao=='Atacante']['id']),
                          random.choice(dados.loc[dados.posicao=='Atacante']['id']),
                          random.choice(dados.loc[dados.posicao=='Atacante']['id']),
                          random.choice(dados.loc[dados.posicao=='Tecnico']['id'])]

        populacao = populacao + geracao
    # Time escolhido    
    time = pd.DataFrame()
    for i in range(12):
        time = time.append(dados[dados.id==melhor[i]])

    if (time.duplicated().any()==True):
        control = 0
    else:
        control = 1

NameError: name 'melhor' is not defined

Score do Time: pontuação entre zero e um.

In [ ]:
time.score_atleta.sum()/12

Preço do Time

In [ ]:
time.preco.sum()

Performance: score de todos os times gerados

In [ ]:
plot(c)

Performance: score dos melhores times ao longo das iterações (convergência)

In [ ]:
plot(a)

O time escalado

In [ ]:
time